In [1]:
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
import copy
from berny import Berny, geomlib,Geometry,Math
import berny.coords
import sys
sys.path.append("../../Libs/")
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FcM_like
from alch_deriv_NEW import alch_deriv
from Morse import mpa, Morse_interpolator
from AP_class import APDFT_perturbator as AP
from FDcoeffs import *
from ABSE import absec
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

In [2]:
%load_ext autoreload
%autoreload 2
from AP_class import APDFT_perturbator as AP

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def nr_step(ic,g,h):
    ncoords=ic.shape[0]*ic.shape[1]
    p1cds=ic-np.linalg.solve((np.swapaxes(h,1,2)).reshape(ncoords,ncoords),g.reshape(-1)).reshape(ic.shape[0],ic.shape[1])
    p1cds-=p1cds[0]
    print (norm(p1cds[1]))
    if p1cds.shape[0]>2:
        print (anglecenter(p1cds),anglecenter(p1cds)*np.pi/180)
    return(p1cds)

In [4]:
def build_h_ic(s,g_ic,h0,B_inv):
    geom0=s.geom.copy()
    B=s.coords.B_matrix(geom0)
    bms=[]
    for i in range(geom0.coords.flatten().shape[0]):
        a=geom0.coords.flatten()
        geom1=geom0.copy()
        a[i]+=.001*bohr2ang
        a=a.reshape(geom0.coords.shape)
        geom1.coords=a
        bms.append((s.coords.B_matrix(geom1)-B)*1000)
    bms_arr=np.asarray(bms)
    BpG2=np.einsum('ijk,j->ik',bms,g_ic)
    h_ic=B_inv.T@(h0-BpG2)@B_inv
    return h_ic

ebde=100/627.5 # empirical bond dissociation energy
def mpa_pb_with_energy(coords,atoml,g,h,e,ghost=[]):
    g=np.asarray(g)
    if not len(h.shape)==2:
        h=h.swapaxes(1,2)
        h=h.reshape(g.shape[0]*3,g.shape[0]*3)
    geom0=Geometry(atoml,coords*bohr2ang)
    bernyobj=Berny(geom0)
    s=bernyobj._state
    B = s.coords.B_matrix(geom0)
    q0=s.coords.eval_geom(geom0)
    B_inv = B.T.dot(Math.pinv(np.dot(B, B.T)))
    g_ic=np.dot(B_inv.T, (g).reshape(-1))

    h_ic=build_h_ic(s,g_ic,h,B_inv)
    bnr=0
    dde_mb=0
    ddq_mb=np.zeros_like(q0)
    for i in range(len(s.coords._coords)):
        if s.coords._coords[i].__class__ is berny.coords.Bond:
            bnr+=1
            if s.coords._coords[i].i not in ghost and s.coords._coords[i].j not in ghost:
                MorseInt=Morse_interpolator(g_ic[i],h_ic[i,i],q0[i],ebde,e)
                ddq_mb[i]+=MorseInt.re+g_ic[i]/h_ic[i,i]-q0[i]
                dde_mb+=MorseInt.e_min-e+1/2*h_ic[i,i]*(MorseInt.re-q0[i])**2
    dq_NR=-np.linalg.solve(h_ic,g_ic)
    ddq_mb[bnr:]=np.linalg.solve(h_ic[bnr:,bnr:],-h_ic[bnr:,:]@(ddq_mb))
    dq_tot=dq_NR+ddq_mb
    en_MB=(e-1/2*np.einsum("i,ij,j",dq_tot,h_ic,dq_tot)+dde_mb)
    #print (s.coords._coords)
    return q0,q0+dq_tot,en_MB
    return q0,dq_NR,q0+dq_NR,q0+dq_tot,en_MB

In [5]:
ch4optg,nh3optg,oh2optg,hfoptg=np.load("optgeoms.npy")
ch4str=("C {} {} {}"+"; H {} {} {}"*4)
ch4str=ch4str.format(*ch4optg.flatten().tolist())
nh3str=("N {} {} {}"+"; H {} {} {}"*3)
nh3str=nh3str.format(*nh3optg.flatten().tolist())
oh2str=("O {} {} {}"+"; H {} {} {}"*2)
oh2str=oh2str.format(*oh2optg.flatten().tolist())
hfstr=("F {} {} {} ; H {} {} {}")
hfstr=hfstr.format(*hfoptg.flatten().tolist())

In [6]:
pcxbs={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6]),'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])\
      ,'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8]),'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])}

In [74]:
CH4=gto.M(atom=ch4str,unit="Bohr",basis=pcxbs,verbose=0)
NH3=gto.M(atom=nh3str,unit="Bohr",basis=pcxbs,verbose=0)
H2O=gto.M(atom=oh2str,unit="Bohr",basis=pcxbs,verbose=0)
HF=gto.M(atom=hfstr,unit="Bohr",basis=pcxbs,verbose=0)

In [8]:
CH4scf=scf.RHF(CH4)
NH3scf=scf.RHF(NH3)
H2Oscf=scf.RHF(H2O)
HFscf=scf.RHF(HF)
NH3scf.scf()

converged SCF energy = -56.2221836146944


-56.22218361469438

In [9]:
phf=scf.RHF(FcM_like(NH3,fcs=[[0,NH3.natm-1],[.1,-.1]]))
phf.scf(dm0=phf.init_guess_by_1e())

/home/administrator/anaconda3/lib/python3.7/site-packages/pyscf/gto/mole.py:1072: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


converged SCF energy = -57.9799514221169


-57.97995142211693

In [17]:
def dzlist(n):
    l=np.zeros(n)
    l[0]=1
    l[-1]=-1
    return l.tolist()
aaff(NH3scf,dzlist(NH3.natm))

array([[-1.64642139,  0.50096783, -0.88077792],
       [-0.0701682 ,  0.08371881,  0.24616194],
       [ 0.16350338,  0.14946154, -0.15315822],
       [ 1.5530862 , -0.73414818,  0.7877742 ]])

In [34]:
apnh3=AP(NH3scf,sites=[0,3])
apnh3.build_all()

In [35]:
apnh3.APDFT3([1,-1])

-76.04751832745292

In [48]:
apnh3.third_deriv([1,-1])

0.7036559063261678

In [72]:
def e_g_h_predict(mol):
    mf=scf.RHF(mol)
    phf=scf.RHF(FcM_like(mol,fcs=[[0,mol.natm-1],[.1,-.1]]))
    mhf=scf.RHF(FcM_like(mol,fcs=[[0,mol.natm-1],[-.1,.1]]))
    mf.scf()
    #phf.scf(dm0=phf.init_guess_by_1e())
    #mhf.scf(dm0=mhf.init_guess_by_1e())
    g0=mf.Gradients().grad()

In [73]:
e_g_h_predict(NH3)

In [61]:
phf.Gradients()

<bound method class_as_method.<locals>.fn of <pyscf.scf.hf.RHF object at 0x7feecb3626a0>>